In [1]:
import sys
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import mlfoundry as mlf

In [2]:
mlf.login(api_key="OGQ1NDI4ZGMtZGY3My00ZWEyLTg2NGMtZjA3OTQzNDkzZDRiOmZjNjc4Mw==", relogin=True)
client = mlf.get_client()

Writing API key at /home/ec2-user/.mlfoundry/credentials.netrc


In [3]:
class MovieLensDataset(torch.utils.data.Dataset):
    def __init__(self, df, mean_rating=3.58):
        self.df = df
        self.mean_rating = mean_rating
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        x = torch.tensor([row.user_id, row.movie_id], dtype=torch.int32)
        y = torch.tensor(row.rating - self.mean_rating, dtype=torch.float)
        return x, y

In [4]:
train = pd.read_csv("data/ml1m_train.csv")
train_dataset = MovieLensDataset(train)
test = pd.read_csv("data/ml1m_test.csv")
test_dataset = MovieLensDataset(test)

In [5]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_biases = torch.nn.Embedding(n_users, 1)
        self.item_biases = torch.nn.Embedding(n_items,1)
        torch.nn.init.xavier_uniform_(self.user_factors.weight)
        torch.nn.init.xavier_uniform_(self.item_factors.weight)
        self.user_biases.weight.data.fill_(0.)
        self.item_biases.weight.data.fill_(0.)
        
    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item)
        pred += (self.user_factors(user) * self.item_factors(item)).sum(1, keepdim=True)
        return pred.squeeze()
    
    def freeze_users(self):
        self.user_factors.weight.requires_grad = False
        self.user_biases.weight.requires_grad = False
        self.item_factors.weight.requires_grad = True
        self.item_biases.weight.requires_grad = True
        
    def freeze_items(self):
        self.user_factors.weight.requires_grad = True
        self.user_biases.weight.requires_grad = True
        self.item_factors.weight.requires_grad = False
        self.item_biases.weight.requires_grad = False

In [6]:
reviewers = 6041
books = 3953
batch_size = 256

In [7]:
device = torch.device("cpu")

In [8]:
def train_loop(model, train_batch, label_batch, loss_func, optimizer):
    # First pass with users layer freeze
    model.freeze_users()
    prediction = model(train_batch[:,0].to(device), train_batch[:,1].to(device))
    loss = loss_func(prediction, label_batch.to(device))    
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    # First pass with items layer freeze
    model.freeze_items()
    prediction = model(train_batch[:,0].to(device), train_batch[:,1].to(device))
    loss = loss_func(prediction, label_batch.to(device))    
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    return loss.item()

In [9]:
def train(nfactor, learning_rate=0.02, weight_decay=1e-5):
    run = client.create_run(project_name="recommendation-system", run_name=f"matrix-factorization-factor{nfactor}", log_system_metrics=False)
    run.log_params({"nfactor": nfactor, "learning_rate": learning_rate, "weight_decay": weight_decay})
    
    model = MatrixFactorization(reviewers, books, n_factors=nfactor)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
    model.to(device)
    loss_func = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    epoches = 100
    last_test_loss = 10000
    for epoch in range(0, epoches):
        # Training
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        train_count = 1
        train_loss = 0.
        for i, (train_batch, label_batch) in pbar:
            loss = train_loop(model, train_batch, label_batch, loss_func, optimizer)
            train_loss += loss
            train_count += 1
            pbar.set_description(f'Train loss at {epoch} batch {i}: {train_loss/train_count}')

        # Calculate test loss
        pbar = tqdm(enumerate(test_dataloader), total=len(test_dataloader))
        test_loss = 0.
        test_count = 1
        for i,( test_batch, label_batch) in pbar:
            with torch.no_grad():
                prediction = model(test_batch[:,0].to(device), test_batch[:,1].to(device))
                loss = loss_func(prediction, label_batch.to(device))
                test_loss += loss.item()
                test_count += 1
                pbar.set_description(f'Test loss at {epoch} batch {i}: {test_loss/test_count}')
        
        run.log_metrics(
            metric_dict={
                "train_loss": train_loss/train_count,
                "test_loss": test_loss/test_count,
            }, step=epoch
        )
        # Early stopping
        current_test_loss = test_loss/test_count
        if last_test_loss - current_test_loss < 0.01:
            print(f"Stopping Training. Last")
            break
        last_test_loss = current_test_loss
    run.log_model(model=model, framework="pytorch")
    run.end()

In [ ]:
train(nfactor=10)

Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/176/d020293a02f64a9ba70ff2da20679771/
[mlfoundry] 2022-07-06T18:07:07+0000 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-07-06T18:07:07+0000 INFO Run 'cloud/recommendation-system/matrix-factorization-factor10-24' has started.
[mlfoundry] 2022-07-06T18:07:07+0000 INFO Parameters logged successfully


Test loss at 0 batch 326: 1.177852744918044:  42%|████▏     | 327/782 [00:12<00:17, 25.89it/s] 

In [14]:
factors = [20, 30, 50, 75, 100]

In [15]:
for factor in factors:
    train(nfactor=factor)

Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/176/598dc4792eb845f0844b3a6221acb09a/
[mlfoundry] 2022-07-06T15:07:58+0000 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-07-06T15:07:58+0000 INFO Run 'cloud/recommendation-system/matrix-factorization-factor20-18' has started.
[mlfoundry] 2022-07-06T15:07:58+0000 INFO Parameters logged successfully


Test loss at 0 batch 781: 10.590846121387312: 100%|██████████| 782/782 [00:28<00:00, 27.59it/s]


[mlfoundry] 2022-07-06T15:10:30+0000 INFO Metrics logged successfully


Test loss at 1 batch 781: 8.263826126035298: 100%|██████████| 782/782 [00:28<00:00, 27.61it/s]


[mlfoundry] 2022-07-06T15:13:04+0000 INFO Metrics logged successfully


...

Test loss at 42 batch 781: 1.2340370471450104: 100%|██████████| 782/782 [00:28<00:00, 27.60it/s]


[mlfoundry] 2022-07-06T16:56:47+0000 INFO Metrics logged successfully


Test loss at 43 batch 781: 1.224152172966783: 100%|██████████| 782/782 [00:27<00:00, 28.43it/s] 


[mlfoundry] 2022-07-06T16:59:16+0000 INFO Metrics logged successfully
Stopping Training. Last
[mlfoundry] 2022-07-06T16:59:25+0000 INFO Model logged successfully


AttributeError: 'MlFoundryRun' object has no attribute 'close'

1